In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Advanced Fork-Join Example 12 - Complex fork-join with 2 parallel branches
val model = Network("model")

In [ ]:
// Create network
val delay = Delay(model, "Delay1")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val queue3 = Queue(model, "Queue3", SchedStrategy.FCFS)
val queue4 = Queue(model, "Queue4", SchedStrategy.FCFS)
val queue5 = Queue(model, "Queue5", SchedStrategy.FCFS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)

val jobclass1 = ClosedClass(model, "class1", 10, delay, 0)

In [ ]:
// Service configurations
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(2.0))
queue3.setService(jobclass1, Exp(1.0))
queue4.setService(jobclass1, Exp(3.0))
queue5.setService(jobclass1, Exp(0.8))
delay.setService(jobclass1, Exp(0.5))

In [ ]:
// Set routing matrix with complex serial routing
val P = model.initRoutingMatrix()

P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
// Complex serial routing:
P.set(jobclass1, jobclass1, queue1, queue4, 1.0)  // queue1 -> queue4
P.set(jobclass1, jobclass1, queue4, queue5, 1.0)  // queue4 -> queue5  
P.set(jobclass1, jobclass1, queue5, join, 1.0)    // queue5 -> join
P.set(jobclass1, jobclass1, queue2, queue3, 1.0)  // queue2 -> queue3
P.set(jobclass1, jobclass1, queue3, join, 1.0)    // queue3 -> join
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(JMT(model, "seed", 23000))
solvers.add(MVA(model))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This complex serial fork-join model demonstrates:

1. **Asymmetric Fork-Join Structure**:
   - Branch 1: Queue1 → Queue4 → Queue5 (3 stations in series)
   - Branch 2: Queue2 → Queue3 (2 stations in series)
   - Different path lengths create load imbalance

2. **Service Rate Heterogeneity**:
   - Queue1: rate 1.0, Queue2: rate 2.0 (different fork outputs)
   - Queue3: rate 1.0, Queue4: rate 3.0, Queue5: rate 0.8
   - Queue5 has the slowest service rate (0.8)

3. **Closed System Properties**:
   - 10 jobs circulate in the system
   - Higher population than basic examples
   - Complex routing creates bottleneck effects

4. **Performance Characteristics**:
   - **Branch 1 bottleneck**: Queue5 (rate 0.8) likely limits performance
   - **Branch 2 advantage**: Shorter path with faster Queue2 (rate 2.0)
   - **Join synchronization**: Must wait for slower branch

5. **FCFS Scheduling**:
   - All queues use First-Come-First-Served
   - Creates potential for blocking and congestion
   - Different from PS scheduling in basic examples

6. **Serial Dependencies**:
   - Each branch has internal dependencies
   - Jobs must complete all stations in sequence
   - Creates compound queueing effects

The model represents:
- Manufacturing with multi-stage parallel processes
- Software systems with complex parallel workflows
- Assembly lines with different branch complexities
- Distributed computing with heterogeneous resources

Key insight: The bottleneck shifts from simple service rates to path-dependent effects, where the combination of path length and service rates determines overall performance.